In [1]:
from langchain_community.document_loaders import PyPDFLoader, UnstructuredPDFLoader

/Users/hanhtet/Documents/workspace/multi-rag-syspare/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# configurations
pdf = "test/testing_ocr.pdf"

In [15]:
# loader pdf
loader = PyPDFLoader(pdf)
docs = loader.load()
print(f"Loaded {len(docs)} documents from {pdf}")

Loaded 27 documents from test/testing_ocr.pdf


In [4]:
loader_fallback = UnstructuredPDFLoader(pdf, mode="elements", strategy="hi_res")
docs_fallback = loader_fallback.load()
print(f"Loaded {len(docs_fallback)} documents from {pdf} using fallback loader")

Loaded 524 documents from test/testing.pdf using fallback loader


In [16]:
tables = []
tables_fallback = []

texts = [d.page_content for d in docs]
texts_fallback = [d.page_content for d in docs_fallback]

In [17]:
texts[0]

"INTRODUCTION \nThank you for purchasing Yanmar tractor product that has been designed and \nmanufactured based on the Yanmar state-of-the-art technology and rich \nexpertise in developing and manufacturing products. \nHandle the tractor correctly by following the instructions in the operation manual \nso that the tractor will provide long years of reliable and predictable service. \nThe operation manual constitutes an indispensable part of the Yanmar tractor \nproduct. Always keep the operation manual readily accessible. \nCarefully study the operation manual to get familiar with the instructions and \ninformations contained in the operation manual. The instructions and \ninformations are helpful in using the tractor correctly and safely, avoiding \npersonal injury and other accidents during operation and servicing of the tractor. \nWhen using any implement together with the tractor, also carefully study the \noperation manual of the implement so that the operator can use the implemen

In [13]:
texts_fallback[1]

'Thank you for purchasing Yanmar tractor product that has been designed and manufactured based on the Yanmar state-of-the-art technology and rich expertise in developing and manufacturing products.'

In [18]:
print("Compare text count")
print("Texts:", len(texts))
print("Texts fallback:", len(texts_fallback))

Compare text count
Texts: 27
Texts fallback: 524


In [ ]:
import nltk, re
nltk.download("words")

In [ ]:
# Load a set of valid English words
from nltk.corpus import words
valid_words = set(words.words())

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text) # Remove unwanted characters (e.g., special symbols)
    text = re.sub(r'\s+', ' ', text).strip() # Normalize whitespace
    text = ' '.join(word for word in text.split() if word.lower() in valid_words) # Filter out unreadable or meaningless words
    return text

cleaned_texts = [clean_text(t) for t in texts]

In [ ]:
cleaned_texts[0]